In [1]:
import concurrent.futures
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint


domain = "https://www.dangquangwatch.vn"


def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


def get_product_info(url):
    soup = get_soup(url)
    title_name = soup.select_one(".product-subname").text.split(" - ")[0].split(" ")

    name = " ".join(title_name[:-1])

    price = soup.select_one(".price").text.replace("\xa0₫", "")
    product_code = title_name[-1]

    details = {}
    table = soup.select_one(".detail_product_right .depro_right_content").find_all(
        class_="item"
    )
    for row in table:
        list_p = row.find_all(name="p")
        first_name = list_p[0].get_text().strip()
        second_name = list_p[1].get_text().strip()
        details[first_name] = second_name
    return {
        "Mã sản phẩm": product_code,
        "Giá tiền": price,
        **details,
    }


def crawl_data_from_web(all_links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(get_product_info, all_links)
    data_frame_watchs = pd.DataFrame(results)
    return data_frame_watchs


all_page = [
    f"https://www.dangquangwatch.vn/sp/dong-ho-chinh-hang.html?&page={idx_page}"
    for idx_page in range(1, 125)
]
all_links = [
    (domain + a["href"]) for url in all_page for a in get_soup(url).select("a.p_info")
]

data_frame_watchs = crawl_data_from_web(all_links)

data_frame_watchs


,Mã sản phẩm,Giá tiền,Đường kính mặt,Chống nước,Chất liệu mặt,Năng lượng sử dụng,Size dây,Chất liệu dây,Chất liệu vỏ,Kiểu dáng,...,Chế độ bảo hành,Thương hiệu,Đối tượng sử dụng,Tính năng,Độ rộng mắt,Độ rộng sống mũi,Độ dài gọng,Mầu gọng,Mầu mắt,Bảo hành
0,E-3429.195.20.58.25,62.200.000đ,45 mm,5 ATM,Sapphire,Cơ lên cót tay,22 mm,Dây da chính hãng,Thép không gỉ,Đồng hồ nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PA5018H,4.072.000đ,42 mm,5 ATM,Tráng Sapphire,Quartz/Pin,21 mm,Dây da chính hãng,Thép không gỉ,Đồng hồ Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PA5018G,4.072.000đ,42 mm,5 ATM,Tráng Sapphire,Quartz/Pin,21 mm,Dây da chính hãng,Thép không gỉ,Đồng hồ Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PA2022,9.520.000đ,40 mm,5 ATM,Tráng Sapphire,Quartz/Pin,20 mm,Dây da chính hãng,Thép không gỉ,Đồng hồ Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PA5021B,4.781.600đ,42 mm,10 ATM,Tráng Sapphire,Quartz/Pin,21 mm,Dây da chính hãng,Thép không gỉ,Đồng hồ Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090,QQ-C213J202Y,852.000đ,30 mm,3ATM,Kính cứng chịu lực,Quartz/Pin,,Hợp kim thép,Hợp kim thép,Nữ,...,Bảo hành quốc tế 01 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3091,QQ-M153J010Y,1.344.000đ,41 mm,10ATM,Kính cứng chịu lực,Quartz/Pin,,Dây nhựa,Vỏ nhựa,Đồng hồ Nam,...,Bảo hành quốc tế 01 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3092,JL-1-1542B,3.456.000đ,42 mm,10 ATM,hardened crystex crystal (Kính cứng ),Quartz (Điện tử),22 mm,Dây da,Thép không gỉ,Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3093,BC-G12S,5.472.500đ,35 mm,5 ATM,Sapphire,Quartz/Pin,20 mm,Thép không gỉ mạ Demi PVD,Thép không gỉ mạ Demi PVD,Nam,...,Bảo hành quốc tế 02 năm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
